# Cosas por agregar
- Sección de series de tiempo (datetime)

# Import importantes

In [1]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime


# Download latest version
kagglehub.dataset_download("isathyam31/adult-income-prediction-classification")


/home/capitanespiral/Documents/GitHub/data_science/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/capitanespiral/.cache/kagglehub/datasets/isathyam31/adult-income-prediction-classification/versions/1'

# Gathering data

## Datos en general

La función más usada, lejos es `read_csv` de pandas, donde basta que demos el path del archivo y su nombre, y se leerá si no hay mayor problema. Por default, la primera fila se transforma en las columnas del dataframe, que podemos ver usando `dataframe.columns`

In [37]:
path="/home/capitanespiral/Documents/GitHub/data_science" #Aquí introducir tu path
filename="data.csv"

df=pd.read_csv(f"{path}/{filename}")
#Display entrega mucho mejor resultado que print para los dataframes!
display(df)
print(df.columns)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'country', 'salary'],
      dtype='object')


---
Opciones más usadas de `read_csv`:
- **sep o delimiter** &rarr; El separador de la data, usualmente se encuentra solo pero a veces es mejor darlo explícitamente. (en el caso que se confunda el identificador automático)
- **header** &rarr; Número de fila que se toma como nombre de columnas y desde el cual comienza la data, por default, cero (inicio del .csv).
- **skiprows** &rarr; Filas a saltar desde el inicio del archivo (indexeando desde cero, se puede entregar lista o tupla)
- **skipfooter** &rarr; Filas a saltar desde el final del archivo.
- **usecols** &rarr; Subset de columnas a usar (secuencia de números o nombres explícitos).

In [7]:
#Si la primera fila está mala, cambiamos el header
df1=pd.read_csv(f"{path}/data_bad_first_row.csv") #Salió pésimo
df2=pd.read_csv(f"{path}/data_bad_first_row.csv",header=1) #Mucho mejor

#Si la segunda está mala
df3=pd.read_csv(f"{path}/data_bad_second_row.csv") #Primer dato pésimo
df4=pd.read_csv(f"{path}/data_bad_second_row.csv",skiprows=[1],header=0) #Nos saltamos la segunda fila y conservamos el header

#Seleccionemos ciertas columnas no más
df5=pd.read_csv(f"{path}/data_bad_first_row.csv",header=1,usecols=[0,3]) #Puede ser con números, aquí la primera y la cuarta
df6=pd.read_csv(f"{path}/data_bad_first_row.csv",header=1,usecols=range(4)) #Puede ser con iteradores, acá me entrega de la primera A la cuarta
df7=pd.read_csv(f"{path}/data_bad_first_row.csv",header=1,usecols=["age","education","workclass"]) #Puede ser con los nombres explícitos

In [8]:
del df1,df2,df3,df4,df5,df6,df7

## Series de Tiempo

Para el caso de series de tiempo, hay que saber trabajar con `Timestamps`, `Datetime`, `Timedelta`, etc...

### Funciones generales 

- **pd.to_datetime()** &rarr; Transforma lo que le entregues en un "Datetime", funciona aceptando varios formatos y si le entregas secuencias.
- **pd.date_range()** &rarr; Para crear puntos en el tiempo equiespaceados, con un "start" un "end", posibilidad de "periods" o "freq"
    - **freq** más comunes:"y" (años),"m" (meses),"W" (semana), "B" (business day), "d" (días), "h" (horas), "min" (minutos), "s" (segundos) Ojo que hay *infinitas* opciones! revisar en https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
- **df.resample().func()** &rarr; Entregando como variable una frecuencia, podemos "resamplear" según la función "func"
- **datetime.min() y .max()** &rarr; Evidente
- **datetimeindex.atributos** &rarr; nos permite acceder a muchas funciones de tiempo como "hour","day","minute","second","dayofyear", "strftime", entre otras (si es una serie, no un índice, agregar dt) 

In [39]:
##datetime con varios formatos
dt1=pd.to_datetime(["13/1/2018", np.datetime64("2018-01-13"), datetime.datetime(2018, 1, 13)],dayfirst=True)
print("dt1:",dt1,"\n")

##date_range
#Creando una lista de tiempo cada tres horas
t1="1-1-2000"
t2="2000-3-10"
dt2=pd.date_range(start=t1,end=t2,freq="3h")
print("dt2:",dt2,"\n")

#Lo mismo pero con 15 minutos
t1="1-1-2000"
t2="2000-10-3"
dt3=pd.date_range(start=t1,end=t2,freq="15min")
print("dt3:",dt3,"\n")

#Si tengo periodos y frecuencia
dt4=pd.date_range("2018-01-01", periods=5, freq="d")
print("dt4:",dt4,"\n")

#Como usar el resampleo
idx = pd.date_range("2018-01-01", periods=10, freq="h")
ts = pd.Series(range(len(idx)), index=idx)
print("ts",ts,"\n")
#Downsample (también muy usado el sum)
ts_downsampled=ts.resample("2h").mean()
print("ts_downsampled",ts_downsampled,"\n")
#upsample (también muy usado bfill)
ts_upsampled=ts.resample("30min").ffill()
print("ts_upsampled",ts_upsampled,"\n")

dt1: DatetimeIndex(['2018-01-13', '2018-01-13', '2018-01-13'], dtype='datetime64[ns]', freq=None) 

dt2: DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 03:00:00',
               '2000-01-01 06:00:00', '2000-01-01 09:00:00',
               '2000-01-01 12:00:00', '2000-01-01 15:00:00',
               '2000-01-01 18:00:00', '2000-01-01 21:00:00',
               '2000-01-02 00:00:00', '2000-01-02 03:00:00',
               ...
               '2000-03-08 21:00:00', '2000-03-09 00:00:00',
               '2000-03-09 03:00:00', '2000-03-09 06:00:00',
               '2000-03-09 09:00:00', '2000-03-09 12:00:00',
               '2000-03-09 15:00:00', '2000-03-09 18:00:00',
               '2000-03-09 21:00:00', '2000-03-10 00:00:00'],
              dtype='datetime64[ns]', length=553, freq='3h') 

dt3: DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 00:15:00',
               '2000-01-01 00:30:00', '2000-01-01 00:45:00',
               '2000-01-01 01:00:00', '2000-01-01 01:15:00',
              

In [10]:
del dt1,dt2,dt3,dt4,idx,ts,ts_upsampled,ts_downsampled

In [46]:
df_t=pd.read_csv("biomet1.csv",skiprows=[1],header=0) #Tenemos caso de segunda linea sin sentido
df_t["time_t"]=pd.to_datetime(df_t['date']+" "+df_t['time']) #Lo guardamos en una nueva columna
display(df_t)
#Podemos acceder a propiedades o métodos específicos usando el "dt"
print("Solo el año:",df_t["time_t"].dt.year,"\n",sep="\n")
print("Cambio el formato:",df_t["time_t"].dt.strftime("%Y/%m/%d"),"\n",sep="\n")

,date,time,DOY,Ta,Pa,RH,Td,Tc,Rn,LWin,...,Ts_3_1_1,SWC_1_1_1,SWC_2_1_1,SWC_3_1_1,SHF_1_1_1,SHF_2_1_1,SHF_3_1_1,LE,H,time_t
0,2014-01-01,00:30:00,1.0208,285.499,100523.0,87.5984,NaN,285.499,NaN,NaN,...,285.912,0.580774,0.677433,0.264485,1.899400,0.491621,0.750120,0.987573,2.10820,2014-01-01 00:30:00
1,2014-01-01,01:00:00,1.0416,285.169,100531.0,88.4022,NaN,285.169,NaN,NaN,...,285.869,0.581969,0.677369,0.264282,1.439690,0.225682,0.559402,0.167466,-4.62751,2014-01-01 01:00:00
2,2014-01-01,01:30:00,1.0624,285.089,100526.0,89.6953,NaN,285.089,NaN,NaN,...,285.818,0.583308,0.677512,0.264079,0.982225,-0.032565,0.359773,-0.209549,-25.06920,2014-01-01 01:30:00
3,2014-01-01,02:00:00,1.0833,285.388,100514.0,87.9180,NaN,285.388,NaN,NaN,...,285.768,0.584545,0.677424,0.263918,0.527264,-0.292637,0.154871,1.012810,-19.36180,2014-01-01 02:00:00
4,2014-01-01,02:30:00,1.1041,285.454,100498.0,84.4577,NaN,285.454,NaN,NaN,...,285.730,0.585552,0.677087,0.264359,0.088923,-0.545190,-0.036778,-1.875870,-2.52581,2014-01-01 02:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68987,2018-02-08,21:00:00,39.8748,284.809,100917.0,87.2794,8.53473,284.809,-111.9970,622.180,...,287.100,0.269830,0.324491,0.168710,1.388390,0.480249,-0.708542,0.028853,-10.19220,2018-02-08 21:00:00
68988,2018-02-08,21:30:00,39.8956,284.447,100944.0,81.5375,6.87156,284.447,-63.4305,669.949,...,287.074,0.270332,0.324650,0.169308,1.220350,0.355083,-1.070870,10.821500,-11.88980,2018-02-08 21:30:00
68989,2018-02-08,22:00:00,39.9165,284.535,100961.0,77.7024,6.43326,284.535,-70.1655,664.107,...,287.047,0.270886,0.324739,0.169821,1.017980,0.206502,-1.642580,28.889900,-46.52960,2018-02-08 22:00:00
68990,2018-02-08,22:30:00,39.9373,283.947,100990.0,81.4990,6.62946,283.947,-95.6326,635.117,...,286.994,0.271494,0.324923,0.170493,0.785127,0.043090,-1.978500,1.498780,-16.58690,2018-02-08 22:30:00


Solo el año:
0        2014
1        2014
2        2014
3        2014
4        2014
         ... 
68987    2018
68988    2018
68989    2018
68990    2018
68991    2018
Name: time_t, Length: 68992, dtype: int32


Cambio el formato:
0        2014/01/01
1        2014/01/01
2        2014/01/01
3        2014/01/01
4        2014/01/01
            ...    
68987    2018/02/08
68988    2018/02/08
68989    2018/02/08
68990    2018/02/08
68991    2018/02/08
Name: time_t, Length: 68992, dtype: object




In [ ]:
#Fijamos un nuevo índice
df_t=df_t.set_index("time_t") #Esto elimina la columna "time_t"
#Al fijarlo como índice, ahora podemos accesar simplemente con ".atributo"
display(df_t)
print(df_t.index.day)
print(df_t.index.dayofyear)

,date,time,DOY,Ta,Pa,RH,Td,Tc,Rn,LWin,...,Ts_2_1_1,Ts_3_1_1,SWC_1_1_1,SWC_2_1_1,SWC_3_1_1,SHF_1_1_1,SHF_2_1_1,SHF_3_1_1,LE,H
time_t,,,,,,,,,,,,,,,,,,,,,
2014-01-01 00:30:00,2014-01-01,00:30:00,1.0208,285.499,100523.0,87.5984,NaN,285.499,NaN,NaN,...,286.550,285.912,0.580774,0.677433,0.264485,1.899400,0.491621,0.750120,0.987573,2.10820
2014-01-01 01:00:00,2014-01-01,01:00:00,1.0416,285.169,100531.0,88.4022,NaN,285.169,NaN,NaN,...,286.498,285.869,0.581969,0.677369,0.264282,1.439690,0.225682,0.559402,0.167466,-4.62751
2014-01-01 01:30:00,2014-01-01,01:30:00,1.0624,285.089,100526.0,89.6953,NaN,285.089,NaN,NaN,...,286.431,285.818,0.583308,0.677512,0.264079,0.982225,-0.032565,0.359773,-0.209549,-25.06920
2014-01-01 02:00:00,2014-01-01,02:00:00,1.0833,285.388,100514.0,87.9180,NaN,285.388,NaN,NaN,...,286.368,285.768,0.584545,0.677424,0.263918,0.527264,-0.292637,0.154871,1.012810,-19.36180
2014-01-01 02:30:00,2014-01-01,02:30:00,1.1041,285.454,100498.0,84.4577,NaN,285.454,NaN,NaN,...,286.329,285.730,0.585552,0.677087,0.264359,0.088923,-0.545190,-0.036778,-1.875870,-2.52581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-02-08 21:00:00,2018-02-08,21:00:00,39.8748,284.809,100917.0,87.2794,8.53473,284.809,-111.9970,622.180,...,287.524,287.100,0.269830,0.324491,0.168710,1.388390,0.480249,-0.708542,0.028853,-10.19220
2018-02-08 21:30:00,2018-02-08,21:30:00,39.8956,284.447,100944.0,81.5375,6.87156,284.447,-63.4305,669.949,...,287.505,287.074,0.270332,0.324650,0.169308,1.220350,0.355083,-1.070870,10.821500,-11.88980
2018-02-08 22:00:00,2018-02-08,22:00:00,39.9165,284.535,100961.0,77.7024,6.43326,284.535,-70.1655,664.107,...,287.483,287.047,0.270886,0.324739,0.169821,1.017980,0.206502,-1.642580,28.889900,-46.52960


Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       ...
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8],
      dtype='int32', name='time_t', length=68992)
Index([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       ...
       39, 39, 39, 39, 39, 39, 39, 39, 39, 39],
      dtype='int32', name='time_t', length=68992)


In [49]:
#Ahora cubrimos todo el espacio
ti=df_t.index.min()
tf=df_t.index.max()
time_total=pd.date_range(start=ti,end=tf,freq="30min")
df_t=df_t.reindex(time_total) #Esta función redefine el índice, conservando datos y agregando nans si el nuevo índice coincide con el anterior o si no existe (para cada fila!)
display(df_t)

,date,time,DOY,Ta,Pa,RH,Td,Tc,Rn,LWin,...,Ts_2_1_1,Ts_3_1_1,SWC_1_1_1,SWC_2_1_1,SWC_3_1_1,SHF_1_1_1,SHF_2_1_1,SHF_3_1_1,LE,H
2014-01-01 00:30:00,2014-01-01,00:30:00,1.0208,285.499,100523.0,87.5984,NaN,285.499,NaN,NaN,...,286.550,285.912,0.580774,0.677433,0.264485,1.899400,0.491621,0.750120,0.987573,2.10820
2014-01-01 01:00:00,2014-01-01,01:00:00,1.0416,285.169,100531.0,88.4022,NaN,285.169,NaN,NaN,...,286.498,285.869,0.581969,0.677369,0.264282,1.439690,0.225682,0.559402,0.167466,-4.62751
2014-01-01 01:30:00,2014-01-01,01:30:00,1.0624,285.089,100526.0,89.6953,NaN,285.089,NaN,NaN,...,286.431,285.818,0.583308,0.677512,0.264079,0.982225,-0.032565,0.359773,-0.209549,-25.06920
2014-01-01 02:00:00,2014-01-01,02:00:00,1.0833,285.388,100514.0,87.9180,NaN,285.388,NaN,NaN,...,286.368,285.768,0.584545,0.677424,0.263918,0.527264,-0.292637,0.154871,1.012810,-19.36180
2014-01-01 02:30:00,2014-01-01,02:30:00,1.1041,285.454,100498.0,84.4577,NaN,285.454,NaN,NaN,...,286.329,285.730,0.585552,0.677087,0.264359,0.088923,-0.545190,-0.036778,-1.875870,-2.52581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-02-08 21:00:00,2018-02-08,21:00:00,39.8748,284.809,100917.0,87.2794,8.53473,284.809,-111.9970,622.180,...,287.524,287.100,0.269830,0.324491,0.168710,1.388390,0.480249,-0.708542,0.028853,-10.19220
2018-02-08 21:30:00,2018-02-08,21:30:00,39.8956,284.447,100944.0,81.5375,6.87156,284.447,-63.4305,669.949,...,287.505,287.074,0.270332,0.324650,0.169308,1.220350,0.355083,-1.070870,10.821500,-11.88980
2018-02-08 22:00:00,2018-02-08,22:00:00,39.9165,284.535,100961.0,77.7024,6.43326,284.535,-70.1655,664.107,...,287.483,287.047,0.270886,0.324739,0.169821,1.017980,0.206502,-1.642580,28.889900,-46.52960
2018-02-08 22:30:00,2018-02-08,22:30:00,39.9373,283.947,100990.0,81.4990,6.62946,283.947,-95.6326,635.117,...,287.444,286.994,0.271494,0.324923,0.170493,0.785127,0.043090,-1.978500,1.498780,-16.58690


# See data and basic statistics

## Indexar

Primero entender que *pandas* tiene dos grandes tipos de objetos: `Series`y `DataFrame`. El primero es como un "vector" (unidimensional) y el segundo está compuesto por varias `Series`, siendo como una "matriz". Y cada uno siempre tiene asociado su `Index`, que puede ser numérico o de cualquier naturaleza (uno común, temporal).

La forma básica de indexar cada tipo tiene distintos resultados:
- **Series[label]** &rarr; Dará un valor escalar en tal posición acorde al `Index`.
- **DataFrame[colname]** &rarr; Dará una *Serie* correspondiente a tal nombre de columna

In [56]:
#Accedo a una columna, me entrega a una serie, y despues a un elemento en específico
print("**Dataframe**")
print("Edad",df["age"],"\n",sep="\n")
print("Tercer valor:",df["age"][2],"\n")

#Si el índice no es numérico, esto igual se puede hacer, pero se recomienda indexar acorde al índice
print("**Dataframe temporal**")
print("Calor sensible",df_t["H"],"\n",sep="\n") #Notar que el índice es temporal
print("Tercer valor indexando con 'int':",df_t["H"][2]) #Obtengo el tercer valor -> Notar advertencia de pandas!
print("Tercer valor indexando con un 'label' del 'index':",df_t["H"]["2014-01-01 01:30:00"]) #Lo mismo pero con el tiempo, engorroso pero preciso

**Dataframe**
Edad
0        39
1        50
2        38
3        53
4        28
         ..
32556    27
32557    40
32558    58
32559    22
32560    52
Name: age, Length: 32561, dtype: int64


Tercer valor: 38 

**Dataframe temporal**
Calor sensible
2014-01-01 00:30:00     2.10820
2014-01-01 01:00:00    -4.62751
2014-01-01 01:30:00   -25.06920
2014-01-01 02:00:00   -19.36180
2014-01-01 02:30:00    -2.52581
                         ...   
2018-02-08 21:00:00   -10.19220
2018-02-08 21:30:00   -11.88980
2018-02-08 22:00:00   -46.52960
2018-02-08 22:30:00   -16.58690
2018-02-08 23:00:00   -22.25020
Freq: 30min, Name: H, Length: 71998, dtype: float64


Tercer valor indexando con 'int': -25.0692
Tercer valor indexando con un 'label' del 'index': -25.0692


/tmp/ipykernel_6844/3667856936.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Tercer valor indexando con 'int':",df_t["H"][2]) #Obtengo el tercer valor -> Notar advertencia de pandas!


Siempre podemos acceder a **varios valores** de una serie (con indexación típica de python) o **varias columnas** (agregando un nuevo []:). Notar que esto último entrega un nuevo `DataFrame` que no permite indexación "típica" de python.

In [57]:
display(df[["age","education"]])
#display(df[["age","education"]][1:5]) #Esto tira error
print("Del segundo al quinto valor de una columna:",df["age"][1:5],sep="\n")

,age,education
0,39,Bachelors
1,50,Bachelors
2,38,HS-grad
3,53,11th
4,28,Bachelors
...,...,...
32556,27,Assoc-acdm
32557,40,HS-grad
32558,58,HS-grad
32559,22,HS-grad


Del segundo al quinto valor de una columna:
1    50
2    38
3    53
4    28
Name: age, dtype: int64


Hay una serie de funciones que nos permiten visualizar la data. Dentro de las más básicas:
- **df.head(n)** &rarr; nos permite ver las primeras "n" filas (sin n, por default 5), funciona para `Series` y `DataFrame`.
- **df.tail(n)** &rarr; nos permite ver las últimas "n" filas (lo mismo), funciona para `Series` y `DataFrame`.

In [61]:
#Para dataframes
display(df.head())
display(df.head(10))

#Para una serie
print(df["age"].head())

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,0
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,0
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,1
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,1


0    39
1    50
2    38
3    53
4    28
Name: age, dtype: int64


In [62]:
display(df.tail())

display(df["workclass"].tail())

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,1


32556          Private
32557          Private
32558          Private
32559          Private
32560     Self-emp-inc
Name: workclass, dtype: object

# Links de interés
- Datasets https://www.kaggle.com/datasets
- Documentación de Pandas https://pandas.pydata.org/docs/index.html 
- Cookbook de pandas https://pandas.pydata.org/docs/user_guide/cookbook.html
- Estructura y un par de cosas sobre data engineering https://apmonitor.com/pds/index.php/Main/DataPreparation